In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
import tqdm
import time

C:\Users\ali\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ali\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
X = pd.read_pickle('./X1.pkl')
y = pd.read_pickle('./y1.pkl')

In [3]:
for col in X.columns:
#     if X[col].dtype=='float64':
#         print col
    print col,X[col].dtype

affiliate_channel_api uint8
affiliate_channel_content uint8
affiliate_channel_direct uint8
affiliate_channel_other uint8
affiliate_channel_remarketing uint8
affiliate_channel_sem-brand uint8
affiliate_channel_sem-non-brand uint8
affiliate_channel_seo uint8
affiliate_channel_nan uint8
affiliate_provider_Other uint8
affiliate_provider_bing uint8
affiliate_provider_direct uint8
affiliate_provider_facebook uint8
affiliate_provider_google uint8
affiliate_provider_other uint8
affiliate_provider_padmapper uint8
affiliate_provider_nan uint8
first_affiliate_tracked_linked uint8
first_affiliate_tracked_local ops uint8
first_affiliate_tracked_marketing uint8
first_affiliate_tracked_omg uint8
first_affiliate_tracked_product uint8
first_affiliate_tracked_tracked-other uint8
first_affiliate_tracked_untracked uint8
first_affiliate_tracked_nan uint8
first_browser_-unknown- uint8
first_browser_Android Browser uint8
first_browser_Chrome uint8
first_browser_Chrome Mobile uint8
first_browser_Firefox uint8

In [4]:
print X.shape
print y.shape

(69847, 113)
(69847L,)


In [5]:
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    
#     print cm
#     print cr
    print a
    return a

all_models = {}

## KNN

In [28]:
a = evaluate_model(KNeighborsClassifier())

0.561250298258


0.5612502982581723

#### GridSearch

In [32]:
params = {'n_neighbors': range(2,40)}
gsknn = GridSearchCV(KNeighborsClassifier(),
                     params, cv=KFold(len(y), n_folds=3, shuffle=True))

In [33]:
gsknn.fit(X, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=69847, n_folds=3, shuffle=True, random_state=None),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [34]:
gsknn.best_params_

{'n_neighbors': 39}

In [35]:
gsknn.best_score_

0.5986083869028018

In [36]:
evaluate_model(gsknn.best_estimator_)

0.599045573849


0.59904557384872348

In [ ]:
all_models['knn'] = {'model': gsknn.best_estimator_,
                     'score': a}

#### Bagging

In [5]:
from sklearn.ensemble import BaggingClassifier
baggingknn = BaggingClassifier(KNeighborsClassifier())

In [6]:
evaluate_model(baggingknn)

0.553424003818


C:\Users\ali\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.55342400381770462

In [7]:
bagging_params = {'n_estimators': [10, 20],
                  'max_samples': [0.7, 1.0],
                  'max_features': [0.7, 1.0],
                  'bootstrap_features': [True, False]}


gsbaggingknn = GridSearchCV(baggingknn,
                            bagging_params, n_jobs=-1,
                            cv=KFold(len(y), n_folds=3, shuffle=True))

In [8]:
gsbaggingknn.fit(X, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=69847, n_folds=3, shuffle=True, random_state=None),
       error_score='raise',
       estimator=BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20], 'max_samples': [0.7, 1.0], 'bootstrap_features': [True, False], 'max_features': [0.7, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [9]:
gsbaggingknn.best_params_

{'bootstrap_features': True,
 'max_features': 0.7,
 'max_samples': 1.0,
 'n_estimators': 20}

In [10]:
all_models['gsbaggingknn'] = {'model': gsbaggingknn.best_estimator_,
                              'score': evaluate_model(gsbaggingknn.best_estimator_)}

0.598616082081


##                                                         Decision Trees

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
all_models['dt'] = {'model': dt,
                    'score': evaluate_model(dt)}

for i in tqdm.tqdm(range(1000)):
    time.sleep(0.01)

0.522643760439


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 95.13it/s]


#### GridSearch

In [12]:
params = {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}

gsdt = GridSearchCV(dt,
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

gsdt.fit(X, y)
print gsdt.best_params_
print gsdt.best_score_

all_models['gsdt'] = {'model': gsdt.best_estimator_,
                      'score': evaluate_model(gsdt.best_estimator_)}

{'min_samples_split': 2, 'splitter': 'best', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}
0.652841210073
0.655738487235


#### Bagging

In [13]:
gsbaggingdt = GridSearchCV(BaggingClassifier(gsdt.best_estimator_),
                           bagging_params, n_jobs=-1,
                           cv=KFold(len(y), n_folds=3, shuffle=True))

gsbaggingdt.fit(X, y)

print gsbaggingdt.best_params_
print gsbaggingdt.best_score_

all_models['gsbaggingdt'] = {'model': gsbaggingdt.best_estimator_,
                             'score': evaluate_model(gsbaggingdt.best_estimator_)}

{'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 20, 'bootstrap_features': False}
0.655862098587
0.658601765688


## Support Vector Machines

In [ ]:
from sklearn.svm import SVC

svm = SVC()
all_models['svm'] = {'model': svm,
                     'score': evaluate_model(svm)}


#### GridSearch

In [ ]:
params = {'C': [0.01, 0.1, 1.0, 10.0, 30.0, 100.0],
          'gamma': ['auto', 0.1, 1.0, 10.0],
          'kernel': ['linear', 'rbf']}


gssvm = GridSearchCV(svm,
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

gssvm.fit(X, y)
print gssvm.best_params_
print gssvm.best_score_

all_models['gssvm'] = {'model': gssvm.best_estimator_,
                      'score': evaluate_model(gssvm.best_estimator_)}

#### Bagging

In [ ]:
gsbaggingsvm = GridSearchCV(BaggingClassifier(gssvm.best_estimator_),
                           bagging_params, n_jobs=-1,
                           cv=KFold(len(y), n_folds=3, shuffle=True))

gsbaggingsvm.fit(X, y)

print gsbaggingsvm.best_params_
print gsbaggingsvm.best_score_

all_models['gsbaggingsvm'] = {'model': gsbaggingsvm.best_estimator_,
                             'score': evaluate_model(gsbaggingsvm.best_estimator_)}

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

rf = RandomForestClassifier()
all_models['rf'] = {'model': rf,
                    'score': evaluate_model(rf)}


In [ ]:
gsrf = GridSearchCV(RandomForestClassifier(n_jobs=-1),
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

## XGBoost

In [6]:
from xgboost.sklearn import XGBClassifier

In [10]:
params1 = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.25,
    'silent': 1.0,
    'n_estimators': 200
}

In [12]:
xgbst1 = XGBClassifier(**params1).fit(X_train, y_train)

In [13]:
prediction_xgbst1 = xgbst1.predict(X_test)

In [20]:
cm

NameError: name 'cm' is not defined

In [19]:
correct = 0

for i in range(len(prediction_xgbst1)):
    if (y_test.iloc[i] == prediction_xgbst1[i]):
        correct += 1
        
acc = accuracy_score(y_test, prediction_xgbst1)

print('Predicted correctly: {0}/{1}'.format(correct, len(prediction_xgbst1)))
print('Error: {0:.4f}'.format(1-acc))

Predicted correctly: 13769/20955
Error: 0.3429


In [ ]:
prediction_xgbst1.

In [21]:
all_models['xgbst1'] = {'model': xgbst1,
                    'score': evaluate_model(xgbst1)}

C:\Users\ali\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.657074683846


In [22]:
params2 = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.05,
    'silent': 1.0,
    'n_estimators': 750,
    'subsample': 0.7,
    'colsample_bytree':0.7,
    'reg_alpha':0,
    'reg_lambda':1.
}

In [23]:
xgbst2 = XGBClassifier( **params2).fit(X_train, y_train)

In [24]:
prediction_xgbst2 = xgbst2.predict(X_test)

In [25]:
correct = 0

for i in range(len(prediction_xgbst2)):
    if (y_test.iloc[i] == prediction_xgbst2[i]):
        correct += 1
        
acc = accuracy_score(y_test, prediction_xgbst2)

print('Predicted correctly: {0}/{1}'.format(correct, len(prediction_xgbst2)))
print('Error: {0:.4f}'.format(1-acc))

Predicted correctly: 13847/20955
Error: 0.3392


In [26]:
all_models['xgbst1'] = {'model': xgbst2,
                    'score': evaluate_model(xgbst2)}

0.660796945836
